In [76]:
#!unzip '/content/ind_pd_2020_1km_ASCII_XYZ.zip'

In [77]:
import pandas as pd
import numpy as np

#Reading in population distribution data
population_dist = pd.read_csv('/content/out.csv')
population_dist.head(20)

,Unnamed: 0,index,X,Y,Z
0,0,746062,84.102916,27.520417,5.963120
1,1,746063,84.111250,27.520417,14.401941
2,2,746064,84.119583,27.520417,6.269425
3,3,746065,84.144583,27.520417,34.482857
4,4,746066,88.069583,27.520417,27.543198
5,5,746067,88.077916,27.520417,15.926651
6,6,746068,88.086250,27.520417,28.964201
7,7,746069,88.094583,27.520417,41.616508
8,8,746070,88.102916,27.520417,33.844357
9,9,746071,88.111250,27.520417,16.496880


In [78]:
len(population_dist)

182080

In [79]:
#Filter out rows not in the region of analysis


population_dist = population_dist[population_dist['Y'] >= 17.782307] 
population_dist = population_dist[population_dist['Y'] <= 24.106277] 
population_dist = population_dist[population_dist['X'] >= 80.244428] #Longitude
population_dist = population_dist[population_dist['X'] <= 84.395551]
population_dist = population_dist.reset_index()



#Latitude - y

population_dist.head()

,level_0,Unnamed: 0,index,X,Y,Z


In [80]:
len(population_dist)

0

In [81]:
from sklearn.cluster import KMeans
import pandas as pd

hospital_coords = pd.read_csv('/content/popular_hospitals_Chhattisgarh.csv')
hospital_coords['Radius'] = (hospital_coords['Popularity'] / np.max(np.array(hospital_coords['Popularity'])))*2
hospital_coords.head()

,Name,Latitude,Longitude,Popularity,Radius
0,KIMS Superspeciality Hospital,22.078054,82.149831,4989,2.000000
1,Shriram Care Hospital Bilaspur,22.085242,82.130087,217,0.086991
2,Care N Cure Super Speciallity Hospital,22.088938,82.149294,137,0.054921
3,Mahadev Super Speciality Hospital,22.066935,82.149309,89,0.035678
4,Unity Hospital,22.085668,82.150168,61,0.024454


In [82]:
X = []

import math

#5 miles
# One degree of latitude equals approximately 364,000 feet (69 miles)
# I will remove everything within 20 miles of the hospital

for x in range(len(population_dist)):
  close = False
  for hospital in range(len(hospital_coords)):
    if math.sqrt((hospital_coords['Latitude'][hospital] - population_dist['Y'][x])**2 + (hospital_coords['Longitude'][hospital] - population_dist['X'][x])**2) < ((hospital_coords['Radius'][hospital]) * (1 / 69) / 30): # 1/ 69 is for converting from miles to lat, long.
      close = True

  if close:
    continue
  for y in range(int(population_dist['Z'][x])):
    X.append([population_dist['X'][x], population_dist['Y'][x]])

X = np.array(X)

In [83]:
Y = []

import math

# One degree of latitude equals approximately 364,000 feet (69 miles)
# I will remove everything within 20 miles of the hospital

for x in range(len(population_dist)):
  for y in range(int(population_dist['Z'][x] / 50)):
    Y.append([population_dist['X'][x], population_dist['Y'][x]])

Y = np.array(Y)

In [84]:
len(X)

0

In [85]:
len(Y)

0

In [86]:
from sklearn.cluster import KMeans
import numpy as np

num_hospitals = 3

kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

print(kmeans.cluster_centers_)

ValueError: ignored

In [ ]:
kmeans.cluster_centers_

In [ ]:
import folium


m = folium.Map(location=[24.286006+2,83.320216+2], zoom_start=7)

for hospital in kmeans.cluster_centers_: #kmeans.cluster_centers_
    print([hospital[1], hospital[0]])
    marker = folium.Marker(location=[hospital[1], hospital[0]], popup='Proposed Hospital', icon=folium.Icon(color='red', icon_color='white', icon='glyphicon glyphicon-heart-empty')) #popup=company['label']
    marker.add_to(m)
    folium.Circle(location=[hospital[1], hospital[0]], fill_color='red', radius=1570, weight=2, color="red").add_to(m) #Multiplied by 1000 for scaling


for x in range(len(hospital_coords)): #kmeans.cluster_centers_
    marker = folium.Marker(location=[hospital_coords['Latitude'][x], hospital_coords['Longitude'][x]], popup=hospital_coords['Name'][x], icon=folium.Icon(color='blue', icon_color='white', icon='glyphicon glyphicon-heart-empty'))
    marker.add_to(m)
    folium.Circle(location=[hospital_coords['Latitude'][x], hospital_coords['Longitude'][x]], fill_color='blue', radius=hospital_coords['Radius'][x]*1500, weight=2, color="blue").add_to(m)


folium.Rectangle([(17.782307,80.244428), (24.106277,84.395551)], fill_color="green", color="green", weight=1).add_to(m)



m

In [ ]:
m.save("index_Chhattisgarh.html")